# Configurations

<table style="width:90%">
    <tr><td style="text-align:left;"><a href="./2 - Basics.ipynb">Previous (Basics)</a></td><td style="text-align:right;"><a href="./4 - Intro to Workflows.ipynb">Next (Intro to Workflows)</a></td></tr>
</table>

* Configurations are how you tell Parsl what compute resources it is to use
* Configurations are specified in Config objects
* Config objects incorporate several different parts

## Executors

These represent the available compute resources at a machine level.

* **ThreadPoolExecutor** - Use for single machine setups
* **HighThroughputExector** - Use for HPC environments where you need less than 2000 nodes needed for the job.
* **ExtremeScaleExecutor** - Use for HPC environments where you need between 1000 and 8000 nodes

In these you can specify things like maximum number of workers, memory per worker, etc.

## Providers

These represent the interface to the job control system

* LocalProvider
* SlurmProvider
* GoogleCloudProvider
* As well as many others

In these you specify account information, partitions, scheduler options, block configurations, etc.

## Launchers

These create wrappers for your submitted code, e.g. in a Slurm environment the Launcher wraps the submitted code in an SRUN framework. 

## Channels

These represent the route of communication from where you are submitting the job to the compute resource.

* **LocalChannel** - Use this any time you are launching your Parsl job from a submit node or are using the ThreadPoolExecutor.
* **SSHChannel** - Allows for remote execution on machines where you have an SSH key set up.
* **OAuthSSHChannel** - Allows for remote execution on machines where you have Globus OAuth tokens set up.
* **SSHInteractiveLoginChannel** - Allows for remote execution using SSH where a key has not been set up.

```python
from parsl.config import Config
from parsl.executors.threads import ThreadPoolExecutor
from parsl.executors import HighThroughputExecutor
from parsl.providers import SlurmProvider
from parsl.launchers import SrunLauncher
from parsl.app.app import python_app

multi_config = Config(
    executors=[
        ThreadPoolExecutor(
            max_threads=8,
            label='local_exec'
        ),
        HighThroughputExecutor(
            label='htex',
            max_workers=2,
            provider=SlurmProvider(
                partition='secondary',
                worker_init='source ~/myconfig',
                walltime='00:30:00',
                init_blocks=1,
                max_blocks=1,
                launcher=SrunLauncher(overrides='--ntasks-per-node=24')
            )
        )
    ])

@python_app(executors=['local_exec'])
def square(x):
    return x * x

@python_app(executors=['htex'])
def compute(root):
    import time
    time.sleep(10)
    return root * 2

```

## Blocks

Parsl defines a resource abstraction called a **block** as the most basic unit of resources to be acquired from a provider. Depending on the individual compute resources a block may represent a set of cores, a single node, a single allocation request to a scheduler, etc.

<table style="margin-left:0px;">
    <tr><td style="text-align: left;">Single block with one node, executing one task</td><td style="text-align: left;">Single block with one node, executing several tasks</td></tr>
    <tr><td><img src="images/N1_T1.png"/></td><td><img src="images/N1_T4.png"/></td></tr>
    <tr><td style="text-align:left;" colspan=2>Single block with multiple nodes, executing multiple tasks</td></tr>
    <tr><td colspan=2><img src="images/N4_T2.png"/></td></tr>
</table>

There are configuration options that define the block
 * `workers_per_node` - The number of concurrent tasks to run on each node
 * `nodes_per_block` - The number of nodes to request per block
 
### Elasticity

Workload requirements often scale with time, sometimes many nodes running in parallel are needed, followed by a task that requires only a single node (e.g. combining results), which is then followed by a tasks that require several nodes again. Parsl addresses this be dynamically provisionning blocks in response to compute need.

<img src="images/parsl_scaling.gif" width="60%"/>

The configuration options for controlling this behavior are
 * `min_blocks` - The minimum number of blocks to maintain (per executor)
 * `init_blocks` - The initial number of blocks to provision
 * `max_blocks` - The maximum number of blocks to that can be active simultaneously
 
 <P>
    <table style="width:90%">
    <tr><td style="text-align:left;"><a href="./2 - Basics.ipynb">Previous (Basics)</a></td><td style="text-align:right;"><a href="./4 - Intro to Workflows.ipynb">Next (Intro to Workflows)</a></td></tr>
</table>
